# Ice Exapansion in Lake Michigan with ArcPy

“I would like to know how to find the area that the different colors on these ice maps cover. 

The idea is to determine what percent of the southern portion of Lake Michigan is covered by each color in order to calculate the average ice coverage on that part of the lake. 

The attached map is an example of an ice coverage map, and the link is to the website where the map originated; this website also has shapefiles and ASCII files in case either of those formats would be easier to handle.”

Original data from http://www.glerl.noaa.gov/data/pgs/glice/glice.html

In [3]:
import arcpy
import time

In [4]:
# This is the defined study area polygon.

study_area = 'data/study_area.shp'

In [5]:
# Place where shapefile data are stored.  All should be unzipped in a single
# directory.

arcpy.env.workspace = 'data/shp2008'
arcpy.env.overwriteOutput = True

In [6]:
# Get list of all shapefiles in that directory.  These files all start with
# the year (2003 to present), so search for any that start with 2.

fns = arcpy.ListFeatureClasses('2*')

# Show the first 5, for verification that it worked.
print(fns[:5])

[u'20071127.shp', u'20071129.shp', u'20071203.shp', u'20071206.shp', u'20071210.shp']


In [7]:
# Define column headings for output data table
columns = [0,5,10,20,30,40,50,60,70,80,90,95,100]
column_names = [("PRC_" + str(i)) for i in columns]

In [8]:
# Create output data table.  

output_table = arcpy.CreateTable_management('out','out_table.dbf')
arcpy.AddField_management(output_table,"date","LONG")
for c in column_names:
    arcpy.AddField_management(output_table,c,"SHORT")
arcpy.DeleteField_management(output_table,"Field1")

<Result 'd:\\data\\Classes\\Current\\GEOG 493 - Computer Methods and Modeling - Fall 2016\\content\\Unit 11 - ArcPy vs Open Source\\out\\out_table.dbf'>

In [9]:
# Create a template database, filling in date fields with shapefile names.
# All other fields (ice coverage counts) default to 0.
insertCursor = arcpy.da.InsertCursor(output_table,"date")
for fn in fns:
    insertCursor.insertRow([int(fn[0:8])])
del insertCursor

In [10]:
# For each shapefile, read in, perform a clip to the study area, and do a 
# frequency analysis.  A search cursor is used to read in values from the 
# frequency table, and an update cursor is used to write those values to the
# output table.

start_time = time.time()

updateCursor = arcpy.da.UpdateCursor(output_table,column_names)
for fn in fns:
    print fn

    # Perform and save the tabulation
    outfile_name = 'table_' + str(fn[0:8]) + '.csv'
    step_1 = arcpy.Clip_analysis(fn,study_area,"clipped_feature.shp")
    step_2 = arcpy.Frequency_analysis(step_1,outfile_name,"iceconc")

    # Retreive this row in the database
    this_row = updateCursor.next()

    # Update row with new info
    searchCursor = arcpy.da.SearchCursor(step_2,'*')
    for tabulate_row in searchCursor:
        this_count = tabulate_row[1]
        this_bin = tabulate_row[2]
        idx = columns.index(this_bin)
        this_row[idx] = this_count
    print this_row
    updateCursor.updateRow(this_row)
    
        
    del searchCursor

del updateCursor

end_time = time.time()

print('Processing complete.  Total time:', end_time - start_time)

20071127.shp
[15161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071129.shp
[15161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071203.shp
[15161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071206.shp
[14825, 336, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071210.shp
[14834, 327, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071213.shp
[14905, 256, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071217.shp
[13475, 1686, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071220.shp
[13950, 1211, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071224.shp
[15161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071227.shp
[15161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20071231.shp
[15161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20080103.shp
[14372, 619, 0, 170, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20080107.shp
[14727, 434, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20080110.shp
[15161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20080114.shp
[15161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20080117.shp
[15161, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20080121.shp
[12486, 1506, 0, 308, 0, 0, 0, 0, 728, 0, 133, 0, 0]
2008